In [3]:
import os
import glob
import numpy as np
import random
from tqdm import tqdm

# =========================================================
# 0. 設定エリア (環境に合わせて調整してください)
# =========================================================
# ファイルパス設定
BASE_DIR = r"C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet"
TARGET_DIR = os.path.join(BASE_DIR, "stage2")  # 対象のnpyファイルがあるフォルダ

# データ拡張のハイパーパラメータ
N_MASKS_RANGE = (1, 3)        # 1区間・1波形あたりの欠損箇所数 (最小, 最大)
MASK_PCT_RANGE = (0.05, 0.12) # 欠損させる長さの割合 (2%〜8%)
NOISE_STD = 0.01              # 0付近にする際のノイズの標準偏差

# 保存時のファイル名につける接尾辞
# 例: "12345-0012.npy" -> "12345-0012_aug.npy"
# これにより学習コードの "split('-')[0]" ロジックを壊さずに済みます。
SUFFIX = "_aug2"

# =========================================================
# 1. データ拡張関数 (提供されたロジック)
# =========================================================
def augment_waveforms(data, n_masks_range=(1, 2), mask_pct_range=(0.02, 0.08), noise_std=0.01):
    """
    波形データに対して、ランダムな区間を0付近の値に置換（欠損シミュレーション）する関数。
    各特徴量(Row 0-3)とその確信度(Row 4-7)のペアごとに独立して処理を行います。
    """
    augmented = data.copy()
    rows, cols = data.shape
    
    # 1. セグメントの境界を特定 (Row 9 の値が変化する場所)
    row9 = data[9]
    change_indices = np.where(np.diff(row9) != 0)[0] + 1
    segment_boundaries = np.concatenate(([0], change_indices, [cols]))
    
    # 2. 各セグメントに対して処理
    for i in range(len(segment_boundaries) - 1):
        start = segment_boundaries[i]
        end = segment_boundaries[i+1]
        seg_len = end - start
        
        # 極端に短いセグメントはスキップ
        if seg_len < 50:
            continue
            
        # 3. 4つの特徴量・確信度ペア (0/4, 1/5, 2/6, 3/7) それぞれに独立して適用
        for feat_idx in range(4):
            conf_idx = feat_idx + 4
            
            # この波形にいくつの欠損を作るか決定
            n_masks = random.randint(n_masks_range[0], n_masks_range[1])
            
            for _ in range(n_masks):
                # 欠損させる長さを決定 (セグメント長に対する割合)
                pct = random.uniform(mask_pct_range[0], mask_pct_range[1])
                mask_len = int(seg_len * pct)
                mask_len = max(1, mask_len) # 最低1フレーム
                
                # 欠損させる開始位置をランダムに決定
                if seg_len > mask_len:
                    rel_start = random.randint(0, seg_len - mask_len)
                    mask_start = start + rel_start
                    mask_end = mask_start + mask_len
                    
                    # ノイズの生成 (0付近)
                    noise_feat = np.random.normal(0, noise_std, mask_len)
                    noise_conf = np.random.normal(0, noise_std, mask_len)
                    
                    # データに適用
                    augmented[feat_idx, mask_start:mask_end] = noise_feat
                    augmented[conf_idx, mask_start:mask_end] = noise_conf
                    
    return augmented

# =========================================================
# 2. メイン実行処理
# =========================================================
def main():
    if not os.path.exists(TARGET_DIR):
        print(f"Error: Directory not found: {TARGET_DIR}")
        return

    # 全ての .npy ファイルを取得
    all_files = glob.glob(os.path.join(TARGET_DIR, "*.npy"))
    
    # 既に生成されたファイル（_aug がついているもの）は除外
    # これにより、元ファイルだけを種として使用します
    source_files = [f for f in all_files if "_aug" not in f]
    
    print(f"Target Directory: {TARGET_DIR}")
    print(f"Source files found: {len(source_files)}")
    print("-" * 50)

    # ★追加: Seedを変えて複数バージョン作成するループ
    NUM_VERSIONS = 3  # 何パターン作るか
    
    total_generated = 0

    for v in range(NUM_VERSIONS):
        # バージョンごとにシード値を変更
        current_seed = 100 + v
        random.seed(current_seed)
        np.random.seed(current_seed)
        
        # 保存名のサフィックス決定 (例: _aug_v1, _aug_v2, _aug_v3)
        current_suffix = f"_aug_v{v+1}"
        
        print(f"\n>>> Generating Version {v+1} (Suffix: {current_suffix}, Seed: {current_seed})")

        for fpath in tqdm(source_files, desc=f"Ver {v+1}"):
            try:
                # データの読み込み
                data = np.load(fpath)
                
                # データ拡張の適用
                aug_data = augment_waveforms(
                    data, 
                    n_masks_range=N_MASKS_RANGE, 
                    mask_pct_range=MASK_PCT_RANGE, 
                    noise_std=NOISE_STD
                )
                
                # 保存ファイル名の作成
                # 例: 12345-0012.npy -> 12345-0012_aug_v1.npy
                fname = os.path.basename(fpath)
                fname_no_ext = os.path.splitext(fname)[0]
                save_name = f"{fname_no_ext}{current_suffix}.npy"
                save_path = os.path.join(TARGET_DIR, save_name)
                
                # 保存
                np.save(save_path, aug_data)
                total_generated += 1
                
            except Exception as e:
                print(f"[Error] Failed to process {fpath}: {e}")

    print("-" * 50)
    print(f"All Processing Complete!")
    print(f"Total new files generated: {total_generated}")

if __name__ == "__main__":
    main()

Target Directory: C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2
Source files found: 8793
--------------------------------------------------

>>> Generating Version 1 (Suffix: _aug_v1, Seed: 100)


Ver 1: 100%|██████████████████| 8793/8793 [00:17<00:00, 510.01it/s]



>>> Generating Version 2 (Suffix: _aug_v2, Seed: 101)


Ver 2: 100%|██████████████████| 8793/8793 [00:19<00:00, 451.08it/s]



>>> Generating Version 3 (Suffix: _aug_v3, Seed: 102)


Ver 3: 100%|██████████████████| 8793/8793 [00:26<00:00, 334.11it/s]

--------------------------------------------------
All Processing Complete!
Total new files generated: 26379


NameError: name 'os' is not defined